In [8]:
import pandas as pd
from dotenv import load_dotenv
from intensity import extract_complete_trajectory_features
from gender_classifier import classify_gender
import librosa
data_dir = "enhanced_audio/"
raw_dir = "../../../MELD.Raw"

Data_processing

In [9]:
data_df = pd.read_csv(raw_dir+"/train/train_sent_emo.csv")
data_df
# train_data_dir = data_dir+"/train/train_splits/"

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"
...,...,...,...,...,...,...,...,...,...,...,...
9984,10474,You or me?,Chandler,neutral,neutral,1038,13,2,3,"00:00:48,173","00:00:50,799"
9985,10475,"I got it. Uh, Joey, women don't have Adam's ap...",Ross,neutral,neutral,1038,14,2,3,"00:00:51,009","00:00:53,594"
9986,10476,"You guys are messing with me, right?",Joey,surprise,positive,1038,15,2,3,"00:01:00,518","00:01:03,520"
9987,10477,Yeah.,All,neutral,neutral,1038,16,2,3,"00:01:05,398","00:01:07,274"


In [11]:
data_df.drop(columns=['Utterance_ID', 'Dialogue_ID','Sentiment', 'Sr No.', 'Utterance', 'Speaker', 'Episode', 'Season', 'StartTime', 'EndTime'], inplace=True)
data_df

,Sr No.,Emotion,Sentiment,Dialogue_ID,Utterance_ID
0,1,neutral,neutral,0,0
1,2,neutral,neutral,0,1
2,3,neutral,neutral,0,2
3,4,neutral,neutral,0,3
4,5,surprise,positive,0,4
...,...,...,...,...,...
9984,10474,neutral,neutral,1038,13
9985,10475,neutral,neutral,1038,14
9986,10476,surprise,positive,1038,15
9987,10477,neutral,neutral,1038,16


In [13]:
data_df = data_df[data_df['Emotion']=="joy"]
data_df

,Sr No.,Emotion,Sentiment,Dialogue_ID,Utterance_ID
23,26,joy,positive,2,2
31,34,joy,positive,2,10
33,36,joy,positive,2,12
44,48,joy,positive,4,1
51,55,joy,positive,4,8
...,...,...,...,...,...
9938,10428,joy,positive,1034,1
9946,10436,joy,positive,1035,6
9947,10437,joy,positive,1036,0
9972,10462,joy,positive,1038,1


In [17]:
def data_formatter(utt_id, dia_id):
    return f"dia{dia_id}_utt{utt_id}_enhanced.wav"

# Create Video_ID column using vectorized operations instead of loop
data_df['Video_ID'] = data_df.apply(lambda row: data_formatter(str(row['Utterance_ID']), str(row['Dialogue_ID'])), axis=1)
data_df.drop(columns=['Utterance_ID', 'Dialogue_ID','Sentiment', 'Sr No.'], inplace=True)
data_df


/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2959465266.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['Video_ID'] = data_df.apply(lambda row: data_formatter(str(row['Utterance_ID']), str(row['Dialogue_ID'])), axis=1)


,Sr No.,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Video_ID
23,26,joy,positive,2,2,dia2_utt2_enhanced.mp4
31,34,joy,positive,2,10,dia2_utt10_enhanced.mp4
33,36,joy,positive,2,12,dia2_utt12_enhanced.mp4
44,48,joy,positive,4,1,dia4_utt1_enhanced.mp4
51,55,joy,positive,4,8,dia4_utt8_enhanced.mp4
...,...,...,...,...,...,...
9938,10428,joy,positive,1034,1,dia1034_utt1_enhanced.mp4
9946,10436,joy,positive,1035,6,dia1035_utt6_enhanced.mp4
9947,10437,joy,positive,1036,0,dia1036_utt0_enhanced.mp4
9972,10462,joy,positive,1038,1,dia1038_utt1_enhanced.mp4


In [27]:
data_df.to_csv("joy_data.csv", index=False)

In [29]:
for row in range(len(data_df)):
    video_path = "enhanced_audio/"+data_df.iloc[row]['Video_ID']
    enhanced, sr = librosa.load(video_path, sr=16000)
    gender = classify_gender(enhanced)
    data_df.iloc[row]['Gender'] = gender
 

Error during gender classification: Audio file not found: [-1.8310547e-04  1.2817383e-03 -1.3427734e-03 ... -3.0517578e-05
  0.0000000e+00  0.0000000e+00]
Error during gender classification: Audio file not found: [-3.6621094e-04 -4.8828125e-04  1.4038086e-03 ...  0.0000000e+00
  3.0517578e-05  0.0000000e+00]
Error during gender classification: Audio file not found: [-4.5776367e-04 -1.8920898e-03 -7.3242188e-04 ...  1.2207031e-04
 -2.7465820e-04 -9.1552734e-05]
Error during gender classification: Audio file not found: [ 3.9672852e-04 -1.0986328e-03  1.4648438e-03 ...  3.0517578e-05
  3.0517578e-05 -3.0517578e-05]
Error during gender classification: Audio file not found: [0.02435303 0.04275513 0.03530884 ... 0.00024414 0.00027466 0.00021362]
Error during gender classification: Audio file not found: [-1.5869141e-03 -5.1879883e-04 -6.4086914e-04 ...  9.1552734e-05
  3.9672852e-04 -1.2817383e-03]
Error during gender classification: Audio file not found: [-6.1035156e-05 -9.1552734e-05  5.187

/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[row]['Gender'] = gender
/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[row]['Gender'] = gender
/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

Error during gender classification: Audio file not found: [-7.6293945e-04 -1.8920898e-03 -6.1035156e-04 ...  3.0517578e-05
  6.1035156e-05  3.0517578e-05]
Error during gender classification: Audio file not found: [ 0.         -0.00067139 -0.00170898 ...  0.          0.
  0.        ]
Error during gender classification: Audio file not found: [-0.00054932 -0.00048828  0.00106812 ... -0.00015259  0.00012207
 -0.00143433]
Error during gender classification: Audio file not found: [ 0.0000000e+00 -1.5258789e-04 -1.5258789e-04 ... -3.0517578e-05
  3.0517578e-05 -4.8828125e-04]
Error during gender classification: Audio file not found: [-4.5776367e-04 -3.0517578e-05 -1.2512207e-03 ... -3.0517578e-04
 -1.8310547e-04 -1.4038086e-03]
Error during gender classification: Audio file not found: [-3.3569336e-04 -7.6293945e-04  1.5258789e-04 ... -3.0517578e-05
  3.0517578e-05  3.0517578e-05]
Error during gender classification: Audio file not found: [-0.00030518 -0.0010376  -0.00097656 ...  0.00088501  0.

/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[row]['Gender'] = gender
/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[row]['Gender'] = gender
/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

Error during gender classification: Audio file not found: [-0.00018311  0.00082397  0.0010376  ... -0.0007019  -0.00057983
 -0.00042725]
Error during gender classification: Audio file not found: [-9.1552734e-05 -5.4931641e-04  2.9602051e-03 ...  1.2207031e-04
  9.1552734e-05  3.0517578e-05]
Error during gender classification: Audio file not found: [-6.7138672e-04  2.3803711e-03  3.2043457e-03 ... -6.1035156e-05
 -3.0517578e-05 -3.0517578e-05]
Error during gender classification: Audio file not found: [-2.7465820e-04 -7.0190430e-04  6.1035156e-05 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00]
Error during gender classification: Audio file not found: [ 1.6052246e-02  2.2003174e-02  1.6540527e-02 ... -4.5776367e-04
 -6.1035156e-05 -1.7395020e-03]
Error during gender classification: Audio file not found: [-2.4414062e-04 -3.3569336e-04  5.4931641e-04 ... -6.1035156e-05
  3.0517578e-04 -1.5258789e-03]
Error during gender classification: Audio file not found: [-1.3183594e-02 -2.6763916e-02

...

/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[row]['Gender'] = gender
/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[row]['Gender'] = gender
/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

Error during gender classification: Audio file not found: [-0.00878906 -0.00567627 -0.00891113 ... -0.00048828 -0.00030518
 -0.00375366]
Error during gender classification: Audio file not found: [-4.6081543e-03 -5.6152344e-03  8.2397461e-04 ... -6.1035156e-05
  4.2724609e-04  6.1035156e-05]
Error during gender classification: Audio file not found: [-0.00036621 -0.00396729 -0.01165771 ... -0.00045776 -0.00015259
 -0.00256348]
Error during gender classification: Audio file not found: [ 0.00036621  0.00497437  0.01037598 ... -0.00027466 -0.00015259
 -0.00036621]
Error during gender classification: Audio file not found: [ 0.00091553  0.0020752   0.00341797 ... -0.00030518  0.00012207
 -0.00241089]
Error during gender classification: Audio file not found: [ 0.00491333  0.00094604 -0.00360107 ...  0.00018311 -0.00024414
 -0.00033569]
Error during gender classification: Audio file not found: [-4.8828125e-04 -5.1879883e-04  6.1035156e-04 ... -3.0517578e-05
  6.1035156e-05  6.1035156e-05]
Error

/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[row]['Gender'] = gender
/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[row]['Gender'] = gender
/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

Error during gender classification: Audio file not found: [ 9.1552734e-04  2.0141602e-03  2.2583008e-03 ...  3.0517578e-04
  3.0517578e-05 -9.4604492e-04]
Error during gender classification: Audio file not found: [-3.9672852e-04 -8.8500977e-04 -5.4931641e-04 ... -3.0517578e-05
 -3.9672852e-04 -3.3569336e-04]
Error during gender classification: Audio file not found: [-3.0517578e-05  3.0517578e-05 -1.2207031e-04 ... -3.0517578e-05
  0.0000000e+00  0.0000000e+00]
Error during gender classification: Audio file not found: [1.8310547e-04 6.4086914e-04 1.4953613e-03 ... 3.0517578e-05 3.0517578e-05
 3.0517578e-05]
Error during gender classification: Audio file not found: [-4.5776367e-04 -7.0190430e-04  7.3242188e-04 ...  1.2207031e-04
  1.2207031e-04  6.1035156e-05]
Error during gender classification: Audio file not found: [-0.00057983 -0.00079346  0.00149536 ... -0.00137329 -0.00061035
 -0.00482178]
Error during gender classification: Audio file not found: [-4.5776367e-04 -1.5258789e-04  2.13

/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[row]['Gender'] = gender
/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[row]['Gender'] = gender
/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2297341177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [30]:
data_df

,Emotion,Video_ID
23,joy,dia2_utt2_enhanced.wav
31,joy,dia2_utt10_enhanced.wav
33,joy,dia2_utt12_enhanced.wav
44,joy,dia4_utt1_enhanced.wav
51,joy,dia4_utt8_enhanced.wav
...,...,...
9938,joy,dia1034_utt1_enhanced.wav
9946,joy,dia1035_utt6_enhanced.wav
9947,joy,dia1036_utt0_enhanced.wav
9972,joy,dia1038_utt1_enhanced.wav
